In [1]:
with open("data/the-verdict.txt", 'r') as f:
    raw = f.read()

In [2]:
len(raw)

20479

In [3]:
raw[:40]

'I HAD always thought Jack Gisburn rather'

In [6]:
import re

txt = "Hello, kid. How are you doing?"
res = re.split(r"(\s)", txt)

res

['Hello,', ' ', 'kid.', ' ', 'How', ' ', 'are', ' ', 'you', ' ', 'doing?']

In [8]:
res = re.split(r'([,.!:;?_"()\']|--|\s)', txt)
res = [item.strip() for item in res if item.strip()]
res

['Hello', ',', 'kid', '.', 'How', 'are', 'you', 'doing', '?']

In [9]:
preprocessed = re.split(r'([,.!:;?_"()\']|--|\s)', raw)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
preprocessed[:15]

['I',
 'HAD',
 'always',
 'thought',
 'Jack',
 'Gisburn',
 'rather',
 'a',
 'cheap',
 'genius',
 '--',
 'though',
 'a',
 'good',
 'fellow']

In [10]:
len(preprocessed)

4690

In [12]:
# token id assignment
total_words = sorted(set(preprocessed))
vocab_size = len(total_words)

vocab_size

1130

In [14]:
vocab = {token:integer for integer, token in enumerate(total_words)}

for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 20:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)


In [17]:
# tokenizer class - encode and decode

class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.!:;?_"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'([,.!:;?_"()\']|--|\s)', r'\1', text)
        return text


In [19]:
tokenizer = SimpleTokenizerV1(vocab)

text = "It's the last he painted, you know!"
ids = tokenizer.encode(text)
ids

[56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 0]

In [20]:
tokenizer.decode(ids)

"It ' s the last he painted , you know !"

In [21]:
text = "how are you?"
tokenizer.encode(text)
# out of vocabulary keywords like "hello"

[560, 169, 1126, 10]

In [22]:
total_tokens = sorted(list(set(preprocessed)))
total_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer, token in enumerate(total_tokens)}

In [23]:
len(vocab.items())

1132

In [24]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [30]:
# tokenizer updated class - encode and decode

class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.!:;?_"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'([,.!:;?_"()\']|--|\s)', r'\1', text)
        return text


In [31]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello bro what's up!"
text2 = "Sunlight is mesmerising."

text = " <|endoftext|> ".join((text1, text2))
text

"Hello bro what's up! <|endoftext|> Sunlight is mesmerising."

In [32]:
tokenizer.encode(text)

[1131, 1131, 1089, 2, 850, 1051, 0, 1130, 1131, 584, 1131, 7]

In [33]:
tokenizer.decode(tokenizer.encode(text))

"<|unk|> <|unk|> what ' s up ! <|endoftext|> <|unk|> is <|unk|> ."

In [34]:
# BPE
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

In [40]:
text = "Hello bro what's up! <|endoftext|> Sunlightismesmerising."

ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
ids

[15496,
 1379,
 644,
 338,
 510,
 0,
 220,
 50256,
 3825,
 2971,
 1042,
 274,
 647,
 1710,
 13]

In [41]:
tokenizer.decode(ids)

"Hello bro what's up! <|endoftext|> Sunlightismesmerising."

In [42]:
# creating input-target pairs

raw[:50]

'I HAD always thought Jack Gisburn rather a cheap g'

In [43]:
enc_text = tokenizer.encode(raw)
len(enc_text)

5145

In [45]:
sample = enc_text[:50]
sample[:5]

[40, 367, 2885, 1464, 1807]

In [48]:
context_size = 4

x = sample[:context_size]
y = sample[1:context_size + 1]


In [49]:
print(f"X: {x}")
print(f"Y:     {y}")

X: [40, 367, 2885, 1464]
Y:     [367, 2885, 1464, 1807]


In [50]:
for i in range(1, context_size+1):
    ip = sample[:i]
    t = sample[i]
    print(f"{ip} --> {t}")

[40] --> 367
[40, 367] --> 2885
[40, 367, 2885] --> 1464
[40, 367, 2885, 1464] --> 1807


In [ ]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_len, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

        for i in range(0, len(token_ids) - max_len, stride):
            ip_chunk = token_ids[i : i + max_len]
            target_chunk = token_ids[i + 1 : i + max_len + 1]
            self.input_ids.append(torch.tensor(ip_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    def 